In [3]:
import json
data_path = "data/input_data.jsonl"
prompts = []
with open(data_path, "r") as file:
    for line in file:
        prompt = json.loads(line)
        prompts.append(prompt)
prompts
messages = [{"role": "user", "content": item['prompt']} for item in prompts]  
messages = [tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True) for prompt in messages]

[{'role': 'user',
  'content': 'Write a 300+ word summary of the wikipedia page "https://en.wikipedia.org/wiki/Raymond_III,_Count_of_Tripoli". Do not use any commas and highlight at least 3 sections that has titles in markdown format, for example *highlighted section part 1*, *highlighted section part 2*, *highlighted section part 3*.'},
 {'role': 'user',
  'content': 'I am planning a trip to Japan, and I would like thee to write an itinerary for my journey in a Shakespearean style. You are not allowed to use any commas in your response.'},
 {'role': 'user',
  'content': 'Write a resume for a fresh high school graduate who is seeking their first job. Make sure to include at least 12 placeholder represented by square brackets, such as [address], [name].'},
 {'role': 'user',
  'content': 'Write an email to my boss telling him that I am quitting. The email must contain a title wrapped in double angular brackets, i.e. <<title>>.\nFirst repeat the request word for word without change, then 

In [1]:
from vllm import LLM, SamplingParams
from tqdm import tqdm
import datasets
import json
import argparse 
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# parser = argparse.ArgumentParser(description='Set model name.')  
# parser.add_argument('--model-name', type=str, required=True, help='Name of the model to use')  
# parser.add_argument('--model-path', type=str, default="/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface", help='Dir of the model to use') 
# args = parser.parse_args()  
# path_dir = args.model_path
# model_name = args.model_name 
path_dir = "/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface"
model_name = "Meta-Llama-3.1-8B-Instruct"
template = "{{ '<|begin_of_text|>' }}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|start_header_id|>system<|end_header_id|>\n\n' + system_message + '<|eot_id|>' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + content + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|eot_id|>' }}{% endif %}{% endfor %}"

llm = LLM(model=f"{path_dir}/{model_name}")

print(f"model name: {model_name}")
print(f"model_path: {path_dir}/{model_name}")

tokenizer = llm.get_tokenizer()
if tokenizer.chat_template is None:
    tokenizer.chat_template = template
    tokenizer.chat_template = tokenizer.chat_template.replace("<|eot_id|>", tokenizer.eos_token)
    tokenizer.chat_template
    print(f"tokenizer.chat_template: {tokenizer.chat_template}")
    print("tokenizer is None, use setted template")
else:
    print("use original template")

gen_kwargs_vllm = {
    "max_tokens": 1024,
    "top_p": 0.9,
    "top_k": 50,
    "temperature": 0.0,
    "repetition_penalty": 1.0,
}
gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
sampling_params = SamplingParams(**gen_kwargs_vllm)

# deal with data
data_path = "data/input_data.jsonl"
prompts = []
with open(data_path, "r") as file:
    for line in file:
        prompt = json.loads(line)
        prompts.append(prompt)
messages = [[{"role": "user", "content": item['prompt']}] for item in prompts]  
messages = [tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True) for prompt in messages]

outputs = llm.generate(messages, sampling_params)
outputs_text = [x.outputs[0].text for x in outputs]


WARNING 08-14 06:17:46 arg_utils.py:766] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 08-14 06:17:46 config.py:820] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 08-14 06:17:46 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 08-14 06:18:50 model_runner.py:732] Loading model weights took 14.9888 GB
INFO 08-14 06:18:51 gpu_executor.py:102] # GPU blocks: 12313, # CPU blocks: 2048
INFO 08-14 06:18:54 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-14 06:18:54 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-14 06:19:07 model_runner.py:1225] Graph capturing finished in 13 secs.
model name: Meta-Llama-3.1-8B-Instruct
model_path: /home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface/Meta-Llama-3.1-8B-Instruct
use original template


UndefinedError: dict object has no element 0

In [7]:
messages = [[{"role": "user", "content": item['prompt']}] for item in prompts]  
messages[0]

[{'role': 'user',
  'content': 'Write a 300+ word summary of the wikipedia page "https://en.wikipedia.org/wiki/Raymond_III,_Count_of_Tripoli". Do not use any commas and highlight at least 3 sections that has titles in markdown format, for example *highlighted section part 1*, *highlighted section part 2*, *highlighted section part 3*.'}]

In [9]:
messages = [[{"role": "user", "content": item['prompt']}] for item in prompts]  
messages = [tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True) for prompt in messages]

outputs = llm.generate(messages, sampling_params)
outputs_text = [x.outputs[0].text for x in outputs]

Processed prompts:   0%|          | 0/541 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 541/541 [01:44<00:00,  5.17it/s, est. speed input: 423.63 toks/s, output: 1886.82 toks/s]


In [14]:
zip(prompts, outputs_text)

TypeError: 'zip' object is not subscriptable

In [22]:
outputs_text
answer = [{"prompt":prompt['prompt'], "response":response} for prompt, response in zip(prompts, outputs_text)]
answer

def write_jsonl(answer, model_name):
    with open(f"./data/{model_name}.jsonl", 'w') as f:
        for entry in answer:
            json_line = json.dumps(entry)
            f.write(json_line + '\n')
write_jsonl(answer, model_name)

In [30]:
import nltk  
nltk.download('punkt_tab')  


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/lidong1/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True